In [ ]:
#add tools.  Ipopt is a nonlinear solver, JuMP is a framework to simplify coding/math.
using JuMP, Ipopt, Gadfly

In [ ]:
#create model
mod = Model(solver=IpoptSolver())

In [ ]:
# Constants
p_0 = 0    # Initial position [m]
p_f = 100  # Ending position [m] (this is the distance over which we'll minimize)
v_0 = 11    # Initial velocity [m/s]
m_0 = 500    # Initial mass [kg] (~Mazda Miata)
g = 9.81    # Gravity [m/s^2]
tq = 216.93   #Engine Torque [N-m] (suped up Mazda Miata, ~160ft-lb)
rw = 0.18   #wheel radius [m] (~14" rim)
rho = 1.22   #air density at sea level [kg/m^3])
C_d = 0.38    #drag coefficient (Mazda Miata)
A = 2.79     #drag cross sectional area [m^2] (Mazda Miata)

F_max = tq/rw     #the max wheel force produced by the engine [N]

# Time steps
n = 800

In [ ]:
# Set up variables and constraints

# Time step with initial guess
@defVar(mod, Δt ≥ 0, start = 1/n)
# driving time
@defNLExpr(t_d, Δt*n)

# State variables
@defVar(mod, v[0:n] ≥ v_0)            # Velocity (min 11m/s cruise control)
@defVar(mod, p_f ≥ p[0:n] ≥ p_0)          # Distance traveled
@defVar(mod, F_tot[0:n] ≥ 0)         #total control input

# driving control via wheel force
@defVar(mod, 0 ≤ F[0:n] ≤ F_max)

# Provide starting solution
# Could have done this at same time as @defVar
for k in 0:n
    setValue(p[k], (p_f))
    setValue(v[k], (v_0)
    setValue(F[k], F_max/2)
    setValue(F_tot[k], F[k]*k)
end

# set the objective.  We want to minimize control input over timeframe
@setObjective(mod, Min, F_tot[n])

# Initial conditions
@addConstraint(mod, v[0] == v_0)
@addConstraint(mod, p[0] == p_0)
@addConstraint(mod, p[n] == p_f)
@addConstraint(mod, F_tot[0] == 0)

# Drag(v) = 0.5 C_d v^2 rho A
@defNLExpr(drag[j=0:n], 0.5*C_d*(v[j]^2)*rho*A
# Grav(p)   = g sin(alpha)
# alpha(p)  = road grade in degrees at position p
@defNLExpr(grav[j=0:n], g*sin(arctan((sin(p[j])-sin(p[j-1]))/Δt)))

# Dynamics
for j in 1:n
    #trapezoidal integration to discretize position
    @addNLConstraint(mod,
        p[j] == p[j-1] + 0.5*Δt*(v[j]+v[j-1]))
    #same, with acceleration
    @addNLConstraint(mod,
        v[j] == v[j-1] + 0.5*Δt*(
            ((F[j  ] - drag[j  ])/m - grav[j  ]) + 
            ((F[j-1] - drag[j-1])/m - grav[j-1]))
    #calculate total control input so far
    @addConstraint(mod, F_tot[j] == F[j]) + F_tot[j-1]
end

In [ ]:
#solve the problem
status = solve(mod);

In [ ]:
#plot results
h_plot = plot(x=(0:n-1)*getValue(Δt),y=getValue(p)[1:n], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Position"))
m_plot = plot(x=(0:n-1)*getValue(Δt),y=getValue(F_tot)[1:n], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Total Effort"))
v_plot = plot(x=(0:n-1)*getValue(Δt),y=getValue(v)[1:n], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Velocity"))
T_plot = plot(x=(0:n-1)*getValue(Δt),y=getValue(F)[1:n], Geom.line,
                Guide.xlabel("Time (s)"), Guide.ylabel("Instantaneous Effort"))
draw(SVG(6inch, 6inch), vstack( hstack(h_plot,m_plot),
                                hstack(v_plot,T_plot)))